# Data Engineering on Light Pollution data


In [1]:
import pandas as pd

# Task: Use the file of URLs from the first assignment to load the data from 2014-2024 and store it all into a single file.

In [2]:
# Load urls file that is text file which was created in Assignment 1
with open("data/gan_urls.txt", "r") as f:
    urls = f.read().splitlines()

In [3]:
# Load all CSVs and combine it into single DataFrame
dataframes = []
for url in urls:
    print(f"Loading {url}...")
    df = pd.read_csv(url, low_memory=False)
    dataframes.append(df)

# Combine all into one DataFrame
df = pd.concat(dataframes, ignore_index=True)

# Remove duplicates and reset the index
df = df.drop_duplicates().reset_index(drop=True)

Loading http://globeatnight.org/documents/926/GaN2024.csv...
Loading http://globeatnight.org/documents/661/GaN2023.csv...
Loading http://globeatnight.org/documents/662/GaN2022.csv...
Loading http://globeatnight.org/documents/663/GaN2021.csv...
Loading http://globeatnight.org/documents/679/GaN2020.csv...
Loading http://globeatnight.org/documents/665/GaN2019.csv...
Loading http://globeatnight.org/documents/666/GaN2018.csv...
Loading http://globeatnight.org/documents/667/GaN2017.csv...
Loading http://globeatnight.org/documents/668/GaN2016.csv...
Loading http://globeatnight.org/documents/669/GaN2015.csv...
Loading http://globeatnight.org/documents/670/GaN2014.csv...
Loading http://globeatnight.org/documents/671/GaN2013.csv...
Loading http://globeatnight.org/documents/672/GaN2012.csv...
Loading http://globeatnight.org/documents/673/GaN2011.csv...
Loading http://globeatnight.org/documents/674/GaN2010.csv...
Loading http://globeatnight.org/documents/675/GaN2009.csv...
Loading http://globeatni

In [4]:
print(df.columns.tolist())


['ID', 'ObsType', 'Latitude', 'Longitude', 'Elevation(m)', 'LocalDate', 'LocalTime', 'UTDate', 'UTTime', 'LimitingMag', 'SQMReading', 'SQMSerial', 'CloudCover', 'Constellation', 'SkyComment', 'LocationComment', 'Country']


In [5]:
df["year"] = pd.to_datetime(df["LocalDate"]).dt.year
df = df[(df["year"] >= 2014) & (df["year"] <= 2024)]


In [6]:
print(df["year"].min(), df["year"].max())

2014 2024


# Task: Merge LocalDate and LocalTime and convert to a datetime object.


In [7]:
# Merge LocalDate and LocalTime into LocalDateTime

df["LocalDateTime"] = pd.to_datetime(df["LocalDate"] + " " + df["LocalTime"])

# Task: Convert categorical fields to numeric ones.

In [8]:
# Convert CloudCover to numeric percentages

cloud_map = {
    "clear": 0,
    "1/4 of sky": 0.25,
    "1/2 of sky": 0.50,
    "over 1/2 of sky": 0.75
}
df["CloudCoverPct"] = df["CloudCover"].map(cloud_map)


# Task: Convert Constellation to numeric binary values (binarize the column).


In [9]:
# Convert the 15 Constellation values to columns which contain a binary value

const_dummies = pd.get_dummies(df["Constellation"], prefix="const")
df = pd.concat([df, const_dummies], axis=1)


# Task: Create 5 binary columns loc_urban, loc_suburban, loc_rural, loc_remote and loc_unknown.

In [10]:
# Process LocationComment into loc_* columns

def assign_location(comment: str) -> str:
    if not isinstance(comment, str):
        return "unknown"
    comment = comment.lower()
    if any(word in comment for word in ["city", "downtown", "urban", "metro", "metropolitan"]):
        return "urban"
    elif any(word in comment for word in ["suburb", "suburban", "residential", "town"]):
        return "suburban"
    elif any(word in comment for word in ["farm", "village", "countryside", "rural", "small town"]):
        return "rural"
    elif any(word in comment for word in ["remote", "mountain", "desert", "forest", "isolated"]):
        return "remote"
    else:
        return "unknown"
# (a) convert all LocationComment to a single string using assign_location(),
# (b) use fillna("unknown") to use the string "unknown" as a placeholder.
df["LocationComment"] = df["LocationComment"].apply(assign_location).fillna("unknown")

# c) use get_dummies() to make the values binary, note: you must prefix the new columns with loc.
loc_dummies = pd.get_dummies(df["LocationComment"], prefix="loc")

# (d) merge the new columns from (c) in the DataFrame using concat().
df = pd.concat([df, loc_dummies], axis=1)


# Task: Convert missing SQMReading values to -1.

In [11]:
# Replace missing SQMReading with -1 ( Instead of removing these rows, we will fill them with an invalid value of -1.)

df["SQMReading"] = df["SQMReading"].fillna(-1)


# Task: Store a new dataset file with a reduced set columns which include our numeric ones.

In [12]:
# Select final 27 columns

cols_to_keep = [
    'Latitude','Longitude','Elevation(m)','LocalDateTime','LimitingMag','SQMReading','CloudCoverPct',
    'const_Bootes','const_Canis Major','const_Crux','const_Cygnus','const_Gemini','const_Grus','const_Hercules',
    'const_Leo','const_None','const_Orion','const_Pegasus','const_Perseus','const_Sagittarius','const_Scorpius',
    'const_Taurus','loc_remote','loc_rural','loc_suburban','loc_unknown','loc_urban'
]

# Create any missing const_* or loc_* columns with default 0
for col in cols_to_keep:
    if col not in df.columns:
        df[col] = 0

# Keep only desired columns
df_working = df[cols_to_keep]


In [13]:
# Save final working dataset

df_working.to_csv("2014_to_2024_gan_data_working.csv", index=False)
print("Full working dataset saved as '2014_to_2024_gan_data_working.csv'")

Full working dataset saved as '2014_to_2024_gan_data_working.csv'


In [14]:
# BONUS: Normalize column names in working dataset.
df_final = df_working.copy()
df_final.columns = (
    df_final.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
)

# Save the final dataset
df_final.to_csv("2014_to_2024_gan_data_final.csv", index=False)